# Symbolブロックチェーンで複数人が同時に送金するトランザクションをPythonで作成する(2)

[Symbolブロックチェーンで複数人が同時に送金するトランザクションをPythonで作成する(2)](https://qiita.com/nem_takanobu/items/7e7bed7e612b613cf296)

In [1]:
# 起案者アカウント(alice)の準備

from binascii import unhexlify
from symbolchain.core.CryptoTypes import PrivateKey
from symbolchain.core.sym.KeyPair import KeyPair
from symbolchain.core.facade.SymFacade import SymFacade

facade = SymFacade('public_test')

b = unhexlify("見せられないよ！")

alicePrikey = PrivateKey(b)
aliceKeypair = KeyPair(alicePrikey)
alicePubkey = aliceKeypair.public_key
aliceAddress = facade.network.public_key_to_address(alicePubkey)
str(aliceAddress)

'TDIFOJ4FFN7F3OF6IRM5IMQ7WCTTFD7BDXDY7FQ'

In [9]:
# 連署アカウント(bob)の準備
# from symbolchain.core.CryptoTypes import PublicKey
# bobPubkey = PublicKey(unhexlify(""))
# bobAddress = SymFacade.Address("")

bobPrikey = PrivateKey.random()
bobKeypair = SymFacade.KeyPair(bobPrikey)
strBobPubkey = str(bobKeypair.public_key)
strBobAddress = str(facade.network.public_key_to_address(bobKeypair.public_key))

In [15]:
# アグリゲートトランザクションの作成と署名

bobPubkey = strBobPubkey
bobAddress = strBobAddress

msg = 'test'
bobTx = facade.transaction_factory.create_embedded({
    'type': 'transfer',
    'signer_public_key': bobPubkey,
    'recipient_address': aliceAddress,
    'message': bytes(1) + msg.encode('utf8')
})

aliceTx = facade.transaction_factory.create_embedded({
    'type': 'transfer',
    'signer_public_key': alicePubkey,
    'recipient_address': bobAddress,
    'message': bytes(1) + msg.encode('utf8')
})

from symbolchain.core.sym.MerkleHashBuilder import MerkleHashBuilder
from symbolchain.core.CryptoTypes import Hash256
import sha3

hash_builder = MerkleHashBuilder()
hash_builder.update(Hash256(sha3.sha3_256(aliceTx.serialize()).digest()))
hash_builder.update(Hash256(sha3.sha3_256(bobTx.serialize()).digest()))
merkle_hash = hash_builder.final()

import datetime
deadline = (int((datetime.datetime.today() + datetime.timedelta(hours=2)).timestamp()) - 1616694977) * 1000

aggregate = facade.transaction_factory.create({
    'type': 'aggregateBonded',
    'signer_public_key': alicePubkey,
    'fee': 1000000,
    'deadline': deadline,
    'transactions_hash': merkle_hash,
    'transactions': [aliceTx,bobTx]
})

signature = facade.sign_transaction(aliceKeypair, aggregate)
aggregate.signature = signature.bytes

AssertionError: required argument bytes(24)